# Import Neccessary Librabies

In [27]:
!wget https://raw.githubusercontent.com/uttej2001/Anomaly-Logs-Detection/main/logs_dataset.zip # Assuming this is the raw link

import pandas as pd
import numpy as np
import datetime as dt
import random

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')
import matplotlib.style as style
style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 8)


from sklearn import preprocessing
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

--2025-07-21 18:48:24--  https://raw.githubusercontent.com/uttej2001/Anomaly-Logs-Detection/main/logs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9433456 (9.0M) [application/zip]
Saving to: ‘logs_dataset.zip.3’

logs_dataset.zip.3  100%[===================>]   9.00M  --.-KB/s    in 0.05s   

2025-07-21 18:48:25 (164 MB/s) - ‘logs_dataset.zip.3’ saved [9433456/9433456]



In [28]:
df=pd.read_csv("logs_dataset.csv")

In [29]:
df

,@timestamp,_id,ip_address
0,"July 8th 2019, 14:43:03.000",XswJ0msBoTGddM7vxMDB,10.1.1.285
1,"July 8th 2019, 14:43:01.000",dKQJ0msB7mP0GwVzvJjz,10.1.2.389
2,"July 8th 2019, 14:42:59.000",CcwJ0msBoTGddM7vtb8y,10.1.1.415
3,"July 8th 2019, 14:42:57.000",bKQJ0msB7mP0GwVzrZdT,10.1.1.79
4,"July 8th 2019, 14:42:55.000",L6QJ0msB7mP0GwVzpZeI,10.1.1.60
...,...,...,...
721542,"June 9th 2019, 00:00:25.000",dkWJOWsBoTGddM7vRUOR,10.1.2.66
721543,"June 9th 2019, 00:00:19.000",TBuJOWsB7mP0GwVzLmol,10.1.1.249
721544,"June 9th 2019, 00:00:15.000",vUWJOWsBoTGddM7vHkGD,10.1.1.200
721545,"June 9th 2019, 00:00:09.000",aUWJOWsBoTGddM7vB0AR,10.1.2.432


# Feature Extraction

In [36]:
df['@timestamp'] = pd.to_datetime(df['@timestamp'], format='mixed', dayfirst=False)

This changes the timestamp from original to modified form

In [31]:
df.sort_values(['ip_address', '@timestamp'], inplace=True)

This sorts the dataset wrt to ip address and timestamp

In [37]:
df['shift_time'] = df.groupby(['ip_address'])['@timestamp'].shift(1)

It creates shift_time column one shiftdown for timestamp coumn

In [38]:
df['time_diff'] = (df['@timestamp'] - df['shift_time']).dt.seconds // 60

Finding the time differnce between timestamp and shift time

In [39]:
df['date'] = df['@timestamp'].dt.date

Extracting the date from timestamp and creating seperate column

In [40]:
df['weekday'] = df['@timestamp'].dt.weekday

Extracting the no of weekdays present in timestamp

In [41]:
df['hour'] = df['@timestamp'].dt.hour

Finding the no.of hours present in timestamp column

In [42]:
df['is_weekend'] = ((df['weekday']==5 |(df['weekday']==6)).astype(int))

Finding how many weekend are present in timestamp

In [43]:
df['hour_bucket'] = df['hour']//4

In [44]:
df

,@timestamp,_id,ip_address,shift_time,time_diff,date,weekday,hour,is_weekend,hour_bucket
261040,2019-07-01 00:48:25,nAMBq2sB7mP0GwVzH8KU,10.1.1.1,NaT,NaN,2019-07-01,0,0,0,0
259207,2019-07-01 02:04:47,UgdHq2sB7mP0GwVzCsoE,10.1.1.1,2019-07-01 00:48:25,76.0,2019-07-01,0,2,0,0
256672,2019-07-01 03:50:25,oDWnq2sBoTGddM7vv6rO,10.1.1.1,2019-07-01 02:04:47,105.0,2019-07-01,0,3,0,0
256381,2019-07-01 04:02:30,lQ6yq2sB7mP0GwVzzxnf,10.1.1.1,2019-07-01 03:50:25,12.0,2019-07-01,0,4,0,1
255058,2019-07-01 04:57:39,oxHlq2sB7mP0GwVzTRqn,10.1.1.1,2019-07-01 04:02:30,55.0,2019-07-01,0,4,0,1
...,...,...,...,...,...,...,...,...,...,...
708493,2019-06-09 18:07:49,ZmtsPWsBoTGddM7v0Pzx,10.1.2.99,2019-06-09 17:45:05,22.0,2019-06-09,6,18,0,4
708317,2019-06-09 18:22:29,B0J6PWsB7mP0GwVzPrF1,10.1.2.99,2019-06-09 18:07:49,14.0,2019-06-09,6,18,0,4
708078,2019-06-09 18:42:25,oW2MPWsBoTGddM7vfjZR,10.1.2.99,2019-06-09 18:22:29,19.0,2019-06-09,6,18,0,4
705001,2019-06-09 22:58:49,LUx3PmsB7mP0GwVzPJcV,10.1.2.99,2019-06-09 18:42:25,256.0,2019-06-09,6,22,0,5


In [45]:
ip_addr ='ip_address'

In [46]:
ip_counts = df.groupby(ip_addr)['@timestamp'].count().reset_index()

Counting how many times the user in logging into the computer wrt to ip address

In [47]:
ip_counts

,ip_address,@timestamp
0,10.1.1.1,1446
1,10.1.1.100,2860
2,10.1.1.101,1465
3,10.1.1.106,1408
4,10.1.1.109,1459
...,...,...
381,10.1.2.86,4307
382,10.1.2.89,2826
383,10.1.2.90,2904
384,10.1.2.95,2868


In [ ]:
ip_counts = ip_counts.rename(columns={'@timestamp':'total_count'})

In [ ]:
daily_counts = df.groupby([ip_addr,'date'])['@timestamp'].count().reset_index()

In [ ]:
daily_counts

In [ ]:
daily_counts = daily_counts.rename(columns={'@timestamp':'daily_counts'})

In [ ]:
daily_counts_avg = daily_counts.groupby(ip_addr).daily_counts.median().reset_index()

Calculating the median of the counts for all the date having the same ip address

In [ ]:
daily_counts_avg.head(5)

In [ ]:
weekend_counts = df.groupby([ip_addr, 'is_weekend'])['@timestamp'].count().reset_index()

In [ ]:
weekend_counts

In [ ]:
weekend_counts = weekend_counts.rename(columns={'@timestamp':'weekend_counts'})

In [ ]:
weekend_counts.head()

calucating no.of logins occured in the weekends and weekdays

In [ ]:
weekend_counts_avg = weekend_counts.pivot_table(index=ip_addr, columns='is_weekend').reset_index([0])

In [ ]:
weekend_counts_avg.head()

In [ ]:
weekend_counts_avg.columns = weekend_counts_avg.columns.droplevel()

In [ ]:
weekend_counts_avg

In [ ]:
weekend_counts_avg.columns = [ip_addr, 'week_day', 'weekend']

In [ ]:
weekend_counts_avg['is_weekend_ratio'] = weekend_counts_avg['week_day']/ weekend_counts_avg['weekend']

finding the ratio between weekday and weekend

In [ ]:
weekend_counts_avg.head()

In [ ]:
lean_weekend_counts_avg = weekend_counts_avg[[ip_addr, 'is_weekend_ratio']]

In [ ]:
lean_weekend_counts_avg.head()

In [ ]:
avg_time_data = df.groupby(ip_addr).agg({'time_diff':['mean','max']}).reset_index()

In [ ]:
avg_time_data.head()

In [ ]:
avg_time_data.columns = avg_time_data.columns.droplevel()

In [ ]:
avg_time_data.columns = [ip_addr, 'td_mean', 'td_max']

calulating the mean and max for time_diff column and labelling as td_mean and td_max

In [ ]:
avg_time_data.head()

## Total Features Set

In [ ]:
mer_1 = ip_counts.merge(daily_counts_avg, on=ip_addr, how='left')

In [ ]:
mer_2 = mer_1.merge(lean_weekend_counts_avg, on=ip_addr, how='left')

In [ ]:
final_dataset = mer_2.merge(avg_time_data, on=ip_addr, how='left')

In [ ]:
final_dataset.head()

In [ ]:
ip_map = final_dataset[ip_addr].to_dict()

In [ ]:
RANDOM_STATE = 30

# KMeans

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
final_dataset.columns

Displaying the columns present in the final_dataset

In [ ]:
requi_feature_cols = ['total_count', 'daily_counts', 'is_weekend_ratio', 'td_mean', 'td_max']

In [ ]:
new_data = final_dataset[requi_feature_cols]

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
data_scaled = min_max_scaler.fit_transform(new_data)
new_data = pd.DataFrame(data_scaled,columns=requi_feature_cols)

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
sns.pairplot(final_dataset[requi_feature_cols])

In [ ]:
sns.pairplot(new_data)

In [ ]:
cluster_num = range(1,15)

In [ ]:
kmeans = [KMeans(n_clusters=i, random_state=RANDOM_STATE).fit(new_data) for i in cluster_num]

In [ ]:
kmeans

In [ ]:
scores = [kmeans[i].score(new_data) for i in range(len(kmeans))]

In [ ]:
scores

In [ ]:
fig, sd = plt.subplots()
sd.plot(cluster_num,scores)
plt.show()

Displays the graph between no of clusters taken (i.e., 15) and scores calculate using kmeans

In [ ]:
kmeans_cluster_model=kmeans[5]

In [ ]:
kmeans_cluster_model

In [ ]:
final_dataset['cluster'] = kmeans_cluster_model.predict(new_data)
final_dataset['cluster'].value_counts()

# Clustering model with TSNE

t-Distributed Stochastic Neighbor Embedding (t-SNE)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, random_state=RANDOM_STATE)

In [ ]:
tsne_result = tsne.fit_transform(new_data)

In [ ]:
tsne_result

In [ ]:
final_dataset['tsne-2d-one'] = tsne_result[:,0]
final_dataset['tsne-2d-two'] = tsne_result[:,1]

In [ ]:
tsne_cluster = final_dataset.groupby('cluster').agg({'tsne-2d-one':'mean','tsne-2d-two':'mean'}).reset_index()

In [ ]:
plt.figure(figsize=(16,10))

sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="cluster",
    palette=sns.color_palette("hls", 6),
    data=final_dataset,
    legend="full",
    alpha=1
)

plt.scatter(x="tsne-2d-one", y="tsne-2d-two", data=tsne_cluster, s=100, c='b')

plt.show()

In [ ]:
centers = kmeans_cluster_model.cluster_centers_

In [ ]:
pts = np.asarray(new_data)

In [ ]:
pts

In [ ]:
total_dist = pd.Series()

In [ ]:
def get_sum_square_distance(data, kmeans_cluster_model):
    centers = kmeans_cluster_model.cluster_centers_
    pts = np.asarray(data[requi_feature_cols])
    total_dist = pd.Series()
    for i in range(len(pts)):
        dist = 0
        for j in range(len(centers)):
            a = np.linalg.norm(pts[i] - centers[j])
            dist += a**2
        total_dist.set_value(i, dist)

    return total_dist

In [ ]:
final_dataset['sum_squa_dist'] = get_sum_square_distance(new_data, kmeans_cluster_model)

In [ ]:
plt.hist(final_dataset['sum_squa_dist'], bins=100)

In [ ]:
cutoff = 6

In [ ]:
final_dataset['anomaly_kmeans'] = (final_dataset['sum_squa_dist'] >= cutoff).astype(int)

In [ ]:
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="anomaly_kmeans",
    data=final_dataset,
    legend="full",
    alpha=1
)

In [ ]:
final_dataset.loc[final_dataset['anomaly_kmeans']==1]

# Isolated Forest

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
outlier_frac = 0.028

In [ ]:
final_dataset.columns

In [ ]:
new_data.head()

In [ ]:
iso_model =  IsolationForest(n_jobs=-1, n_estimators=200, max_features=3, random_state=RANDOM_STATE, contamination=outlier_frac)

In [ ]:
iso_model.fit(new_data)

In [ ]:
final_dataset['anomaly_isolated'] = pd.Series(iso_model.predict(new_data))

In [ ]:
final_dataset['anomaly_isolated'] = final_dataset['anomaly_isolated'].map( {1: 0, -1: 1} )

In [ ]:
final_dataset['anomaly_isolated'].value_counts()

In [ ]:
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="anomaly_isolated",
    data=final_dataset,
    legend="full",
    alpha=1
)

In [ ]:
final_dataset.loc[final_dataset['anomaly_isolated']==1]

# One Class SVM

In [ ]:
from sklearn.svm import OneClassSVM

In [ ]:
svm_model =  OneClassSVM(nu=outlier_frac, degree=2, kernel='rbf')

In [ ]:
new_data_clean = new_data.loc[new_data.total_count <=new_data.total_count.quantile(1-outlier_frac)]

In [ ]:
svm_model.fit(new_data_clean)

In [ ]:
final_dataset['anomaly_svm'] = pd.Series(svm_model.predict(new_data))

In [ ]:
final_dataset['anomaly_svm'] = final_dataset['anomaly_svm'].map( {1: 0, -1: 1} )

In [ ]:
final_dataset['anomaly_svm'].value_counts()

In [ ]:
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="anomaly_svm",
    data=final_dataset,
    legend="full",
    alpha=1
)

In [ ]:
final_dataset.loc[final_dataset['anomaly_svm']==1]

# Performance of the Algorithm

We must manually categorise the data and examine the performance of the individual algos because it is unsupervised.

In [ ]:
total_counts_condition = (final_dataset['total_count'] >= final_dataset['total_count'].quantile(0.90))
daily_counts_condition =  (final_dataset['daily_counts'] >= final_dataset['daily_counts'].quantile(0.90))

In [ ]:
final_dataset['anomaly_manual'] = (total_counts_condition | daily_counts_condition).astype(np.int)

In [ ]:
final_dataset.head()

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix

In [ ]:
def get_sensitivity_specificity(y_true, y_pred):
    cf = confusion_matrix(y_true, y_pred)
    sensitivity = cf[0,0]/(cf[:,0].sum())
    specificity = cf[1,1]/(cf[:,1].sum())
    return sensitivity, specificity

In [ ]:
f1_iso = f1_score(final_dataset['anomaly_manual'],final_dataset['anomaly_isolated'])
acc_iso = accuracy_score(final_dataset['anomaly_manual'],final_dataset['anomaly_isolated'])
roc_iso = roc_auc_score(final_dataset['anomaly_manual'],final_dataset['anomaly_isolated'])
sen_iso, spec_iso = get_sensitivity_specificity(final_dataset['anomaly_manual'],final_dataset['anomaly_isolated'])
met_iso = {
           'f1_score': f1_iso,
           'accuracy': acc_iso,
           'roc_score': roc_iso,
           'sensitivity': sen_iso,
           'specificity': spec_iso
          }

In [ ]:
f1_kmeans = f1_score(final_dataset['anomaly_manual'],final_dataset['anomaly_kmeans'])
acc_kmeans = accuracy_score(final_dataset['anomaly_manual'],final_dataset['anomaly_kmeans'])
roc_kmeans = roc_auc_score(final_dataset['anomaly_manual'],final_dataset['anomaly_kmeans'])
sen_kmeans, spec_kmeans = get_sensitivity_specificity(final_dataset['anomaly_manual'],final_dataset['anomaly_kmeans'])
met_kmeans = {
           'f1_score': f1_kmeans,
           'accuracy': acc_kmeans,
           'roc_score': roc_kmeans,
           'sensitivity': sen_kmeans,
           'specificity': spec_kmeans
          }

In [ ]:
f1_svm = f1_score(final_dataset['anomaly_manual'],final_dataset['anomaly_svm'])
acc_svm = accuracy_score(final_dataset['anomaly_manual'],final_dataset['anomaly_svm'])
roc_svm = roc_auc_score(final_dataset['anomaly_manual'],final_dataset['anomaly_svm'])
sen_svm, spec_svm = get_sensitivity_specificity(final_dataset['anomaly_manual'],final_dataset['anomaly_svm'])
met_svm = {
           'f1_score': f1_svm,
           'accuracy': acc_svm,
           'roc_score': roc_svm,
           'sensitivity': sen_svm,
           'specificity': spec_svm
          }

In [ ]:
metrics = {'isolated_forest': met_iso,
           'kmeans': met_kmeans,
           'svm': met_svm
          }

In [ ]:
all_metrics = pd.DataFrame.from_dict(metrics)

In [ ]:
all_metrics

# Ensemble models for Classification

In [ ]:
from sklearn.preprocessing import LabelEncoder
final_dataset['ip_address'] = LabelEncoder().fit_transform(final_dataset.iloc[:, -1])

In [ ]:
features = ['ip_address', 'total_count', 'daily_counts', 'is_weekend_ratio', 'td_mean', 'td_max']
X = final_dataset[features]
y = final_dataset['anomaly_isolated']

In [ ]:
X, y = final_dataset[features], final_dataset['anomaly_isolated']
X.shape, y.shape

## Baseline Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

rf = OneVsRestClassifier(estimator = GaussianNB())

# Create RepeatedStratifiedKFold cross-validator with 10 folds, 3 repeats and a seed of 1.
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=20)

# Calculating accuracy using `cross_val_score()` with model instantiated, data to fit,
# target variable, 'accuracy' scoring, cross validator, n_jobs=-1, and error_score set to 'raise'
n_scores = cross_val_score(rf, X, y, scoring='f1_weighted', cv=cv, error_score='raise')

# Print mean and standard deviation of n_scores:
print(n_scores)
print('Baseline scores \n mean f1 weighted: %.3f with a %.3f standard deviation in scores ' % (np.mean(n_scores), np.std(n_scores)))

## Stacking Classifier

Stacking classifier takes:
1. estimators: List of baseline classifiers

2. final_estimator: Defined meta classifier

3. cv: Number of cross validations to perform

In [ ]:
def create_stacking_models():
    base_models = list()
    base_models.append(('KNNC', KNeighborsClassifier(n_neighbors = len(np.unique(y))
                                                     , weights = 'distance')
                       ))
    base_models.append(('SVC', SVC(kernel = 'linear'
                                   , class_weight = 'balanced'
                                  , break_ties = True)
                       ))
    base_models.append(('GNB', GaussianNB()))
    base_models.append(('RF', RandomForestClassifier(n_estimators= 200,
                                                   oob_score = True,
                                                   class_weight = "balanced",
                                                   random_state = 20,
                                                   ccp_alpha = 0.1)
                       ))


    meta_model = LogisticRegression()
    final_model = StackingClassifier(estimators = base_models, ##Base estimators which will be stacked together
                                     final_estimator = meta_model,
                                     cv = 5
                                    )
    return final_model

def models_all():
    all_models = dict()
    all_models['KNNC']= KNeighborsClassifier(n_neighbors = len(np.unique(y))
                                             , weights = 'distance')
    all_models['SVC']= SVC(kernel = 'linear'
                           , class_weight = 'balanced'
                          , break_ties = True
                          )
    all_models['RF']= RandomForestClassifier(n_estimators= 200,
                                                   oob_score = True,
                                                   class_weight = "balanced",
                                                   random_state = 20,
                                                   ccp_alpha = 0.15)
    all_models['GNB'] = GaussianNB()
    all_models['Stacking'] = create_stacking_models()
    return all_models

def evaluate_model(model):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
    scores = cross_val_score(model, X, y, scoring='f1_weighted', cv=cv, error_score='raise')
    return scores

In [ ]:
model_results = list()
models = models_all()
names = list()

# Creating a for loop that iterates over each name, model in models dictionary
for name, model in models.items():
    scores = evaluate_model(model)
    model_results.append(scores)
    names.append(name)
#   print(model_results)
    print('>%s %.3f (%.3f) \n' % (name, np.mean(scores), np.std(scores)))

In [ ]:
plt.figure(figsize=(15,5))
fig = sns.boxplot(x=names, y=model_results, showmeans=True)
plt.title("Model Comparisions")
plt.show(fig)

### Conclusion
- With a stacking model, we were able to get an F1 score of 96.7%, which was greater than the target of 81.1%

In [ ]:
!unzip logs_dataset.zip.2